# Advanced Usage

This tutorial showcases advanced functionalities and applications of GETTSIM's interface. For an introductory tutorial see [here](basic_usage.ipynb). The introductory tutorial showcases GETTSIM's two main functions using a minimal working example:

1. `set_up_policy_environment` which loads a policy environment for a specified date.

2. `compute_taxes_and_transfers` which allows you to compute taxes and transfers given a specified policy environment for household or individual observations.

This tutorial dives deeper into the GETTSIM interface to acquaintance you with further useful functionalities. Specifically, this tutorial shows how to navigate the numerous [input and target](../gettsim_objects/input_variables.rst) variables that the package supports as well as how GETTSIM processes them internally using the example of child benefits in the German taxes and transfers system.

In [22]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt

from gettsim import set_up_policy_environment
from gettsim import compute_taxes_and_transfers
from gettsim import plot_dag
from gettsim import create_synthetic_data


## Example: Kindergeld (Child Benefits)    
    
For this tutorial, we will focus on *Kindergeld*, which is a child benefit that can be claimed by parents in Germany. *Kindergeld* can be claimed in different ways and eligibility for families to receive it depends on various variables. For instance, *Kindergeld* can be claimed as a monthly payment but also as a tax credit (*Kinderfreibetrag*) which is more advantageous for higher income groups. Additionally, eligibility depends on factors like the age and work status of children. These factors make it a more complex feature of the German taxes and transfers system than one might initially believe.  

In the following, we will inspect in detail how the German *Kindergeld* is implemented in GETTSIM to showcase further functionalities of the package. To start off, we load a policy environment to work with.

In [23]:
policy_params, policy_functions = set_up_policy_environment("2020")

In [24]:
policy_params["wohngeld"]

{'faktor_berechnungsformel': 1.15,
 'koeffizienten_berechnungsformel': {1: {'a': 0.04,
   'b': 0.00058,
   'c': 0.000118},
  2: {'a': 0.03, 'b': 0.000405, 'c': 8.8e-05},
  3: {'a': 0.02, 'b': 0.00035, 'c': 7.09e-05},
  4: {'a': 0.01, 'b': 0.000313, 'c': 3.68e-05},
  5: {'a': 0, 'b': 0.000276, 'c': 3.59e-05},
  6: {'a': -0.01, 'b': 0.000258, 'c': 3.08e-05},
  7: {'a': -0.02, 'b': 0.000239, 'c': 3.16e-05},
  8: {'a': -0.03, 'b': 0.000212, 'c': 3.16e-05},
  9: {'a': -0.04, 'b': 0.000184, 'c': 3.33e-05},
  10: {'a': -0.06, 'b': 0.000147, 'c': 3.85e-05},
  11: {'a': -0.1, 'b': 0.00011, 'c': 4.53e-05},
  12: {'a': -0.14, 'b': 0.000101, 'c': 5.13e-05}},
 'haushaltsgröße_hhn': {1: 5, 2: 12},
 'bonus_sehr_große_haushalte': {'max_anz_personen_normale_berechnung': 12,
  'bonus_jede_weitere_person': 51},
 'abzug_stufen': {0: 0.0, 1: 0.1, 2: 0.2, 3: 0.3},
 'min_miete': {1: 52,
  2: 64,
  3: 76,
  4: 88,
  5: 99,
  6: 99,
  7: 111,
  8: 123,
  9: 135,
  10: 146,
  11: 180,
  12: 286},
 'min_eink': {

The according policy parameters are saved under the key `kindergeld`.

In [25]:
policy_params["kindergeld"]

{'altersgrenze': {'mit_bedingungen': 25, 'ohne_bedingungen': 18},
 'kindergeld': {1: 204, 2: 204, 3: 210, 4: 235},
 'einkommensgrenze': 8004,
 'stundengrenze': 20,
 'kinderbonus': 300,
 'datum': numpy.datetime64('2020-01-01')}

## DAG Plots for Visualization of the Taxes and Transfers System

To get a better picture of how *Kindergeld* is implemented in GETTSIM and, meanwhile, of the structure of the German taxes and transfers system, we can utilize GETTSIM's visualization capabilities which are concentrated in the function `plot_dag`. This function creates a directed acyclic graph (DAG) for the taxes and transfers system. It offers many different visualization possibilities. The [guide on visualizing the taxes and transfers system](../how_to_guides/visualizing_the_system.ipynb) gives an in depth explanation of the function. 

To figure out which variables are relevant for the child benefit, we plot an according slice of the entire taxes and transfers system implemented in GETTSIM using `plot_dag`. The function was already imported with all other relevant packages at the beginning of this tutorial. To select the relevant plot, we have to define selectors that we can pass as arguments to the function. We can check the possible output variables [here](../gettsim_objects/variables_out.rst) to find the relevant variable name for our application.

In [26]:
selectors = {"type": "ancestors", "node": "kindergeld_m"}

Since we are interested in the child benefits, we select the node `kindergeld_m` and plot its `ancestors`, which are all the nodes `kindergeld_m` directly or indirectly depends on. As the plot below shows, the variable depends on many other nodes and generates a very large DAG. Clicking on a node links to the according function or variable.

In [27]:
plot_dag(functions=policy_functions, selectors=selectors)

FigureWidget({
    'data': [{'hoverinfo': 'none',
              'line': {'color': 'blue', 'width': 0.5},
     …

An alternative way to inspect the variable is by looking at its neighbors in the DAG. This depiction shows the related variables and functions up to two nodes away from `kindergeld_m`. It reveals `descendants` of `kindergeld_m`: `kindergeld_m_tu` and `kindergeld_m_hh`. These variables contain the child benefits on tax unit and household level respectively.

In [28]:
selectors = {"type": "neighbors", "node": "kindergeld_m", "order": 2}
plot_dag(functions=policy_functions, selectors=selectors)

FigureWidget({
    'data': [{'hoverinfo': 'none',
              'line': {'color': 'blue', 'width': 0.5},
     …

## Computing Variables of Interest

Once we have inspected the DAG, we now have an impression of the various input variables and functions that influence our variable  of interest. As a next step, we will load a set of simulated household data and inspect how we can compute the *Kindergeld* using `compute_taxes_and_transfers` and use the function's features and error messages to aid us in this process.

### Simulated Data

We simulate ad dataset using `create_synthetic_data`. It contains information on households with two adults (age 50) and zero, one, or two children which are either 5 or 10 years old. Each household type occurs with the same frequency in the data. 

The household observations are otherwise identical i.e. the households with children are just replications of the child-free households and for each set of adults with a given income, there are two counterparts, a set of adults with one child and and a set of adults with two children. 

The dataset is sorted accordingly. The first observations only contain child-free individuals while the last observations show the two-child households.

In [29]:
data = create_synthetic_data(
    hh_typen=["couple"],
    n_children=[0, 1, 2],
    age_adults=[50, 50],
    age_children=[10, 5],
    heterogeneous_vars={"bruttolohn_m": np.linspace(1500, 8000, 250)},
)

In [30]:
data[["hh_id", "hh_typ", "alter", "kind", "bruttolohn_m"]]

,hh_id,hh_typ,alter,kind,bruttolohn_m
0,0,couple_0_children,50,False,1500.0
1,0,couple_0_children,50,False,0.0
2,1,couple_1_children,50,False,1500.0
3,1,couple_1_children,50,False,0.0
4,1,couple_1_children,10,True,0.0
...,...,...,...,...,...
2245,748,couple_1_children,10,True,0.0
2246,749,couple_2_children,50,False,8000.0
2247,749,couple_2_children,50,False,0.0
2248,749,couple_2_children,10,True,0.0


Adults' monthly gross earnings range between €1,500 and €8,000. It is captured in the variable `bruttolohn_m`. We can use the pandas function [pandas.DataFrame.describe](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.describe.html) to assess the variable in detail.

In [31]:
data.bruttolohn_m.describe()

count    2250.000000
mean     1583.333333
std      2489.916065
min         0.000000
25%         0.000000
50%         0.000000
75%      3118.473896
max      8000.000000
Name: bruttolohn_m, dtype: float64

The columns contain all the input variables needed to compute `kindergeld_m`.

In [32]:
data.columns

Index(['p_id', 'hh_id', 'tu_id', 'hh_typ', 'kind', 'bruttolohn_m', 'alter',
       'weiblich', 'rentner', 'alleinerz', 'wohnort_ost', 'in_priv_krankenv',
       'priv_rentenv_beitr_m', 'in_ausbildung', 'selbstständig', 'hat_kinder',
       'betreuungskost_m', 'sonstig_eink_m', 'eink_selbst_m',
       'eink_vermietung_m', 'kapitaleink_brutto_m', 'bruttokaltmiete_m_hh',
       'heizkosten_m_hh', 'wohnfläche_hh', 'bewohnt_eigentum_hh',
       'arbeitsl_monate_lfdj', 'arbeitsl_monate_vorj', 'arbeitsl_monate_v2j',
       'arbeitsstunden_w', 'bruttolohn_vorj_m', 'geburtstag', 'geburtsmonat',
       'geburtsjahr', 'jahr_renteneintr', 'm_elterngeld',
       'm_elterngeld_mut_hh', 'm_elterngeld_vat_hh', 'behinderungsgrad',
       'mietstufe', 'immobilie_baujahr', 'vermögen_bedürft_hh', 'entgeltp',
       'grundr_bew_zeiten', 'grundr_entgeltp', 'grundr_zeiten', 'priv_rente_m',
       'schwerbeh_g', 'm_pflichtbeitrag', 'm_freiw_beitrag', 'm_mutterschutz',
       'm_arbeitsunfähig', 'm_krank_ab_16

### Using Errors and Warnings

As the DAG and column list above show, a large number of inputs is required to compute child benefits for a family. While the DAG is very useful to understand the structure within GETTSIM behind a variable or function, it might be difficult to infer which inputs exactly are needed in the data to compute a desired output. The function `compute_taxes_and_transfers` thus directly provides multiple mechanisms that help you identify the required input variables to compute certain taxes and transfers.

As shown in the [basic usage tutorial](basic_usage.ipynb), the function requires `data`, one or multiple `targets`, and `policy_params` as well as `policy_functions` to compute taxes and transfers for a given policy environment. 

Since our data set includes all required input columns already, the function does so without problems.

In [33]:
result = compute_taxes_and_transfers(
    data=data, params=policy_params, targets="kindergeld_m", functions=policy_functions
)
result.head(3)

,hh_id,tu_id,p_id,kindergeld_m
0,0,0,0,0.0
1,0,0,1,0.0
2,1,1,2,0.0


#### Error Messages: Missing Inputs

However, if we have failed to add a required column, the function throws an error with a message that specifies which columns are missing. For example, the variable `bruttolohn_m` holds information on monthly gross earnings and is required to compute child benefits. Dropping it from the data triggers the error.

In [34]:
incomplete_data = data.drop("bruttolohn_m", axis=1)
result = compute_taxes_and_transfers(
    data=incomplete_data,
    params=policy_params,
    targets="kindergeld_m",
    functions=policy_functions
)

ValueError: The following data columns are missing.

[
    "bruttolohn_m",
]


Similarly, we can pass an empty pandas.DataFrame to the function to get a list of all the necessary input columns to compute the desired target(s).

In [35]:
result = compute_taxes_and_transfers(
    data=pd.DataFrame(),
    params=policy_params,
    targets="kindergeld_m",
    functions=policy_functions,
)

ValueError: The input data must contain the column p_id

#### Error Messages and Warnings: Unused Inputs

The function `compute_taxes_and_transfers` also has an option that allows you to check for unused inputs in your data. This functionality is controlled through the argument `check_minimal_specification`. By default, it is set to `ignore`, meaning no check is conduced. However, it can also be set to `warn` to trigger a warning or `raise` an error that includes a message stating the unused inputs.

In [36]:
result = compute_taxes_and_transfers(
    data=data,
    params=policy_params,
    targets="kindergeld_m",
    functions=policy_functions,
    check_minimal_specification="raise",
)

ValueError: The following columns in 'data' are unused.


[
    "m_elterngeld",
    "grundr_entgeltp",
    "immobilie_baujahr",
    "vermögen_bedürft_hh",
    "bruttolohn_vorj_m",
    "sonstig_eink_m",
    "m_elterngeld_vat_hh",
    "arbeitsl_monate_v2j",
    "bewohnt_eigentum_hh",
    "m_elterngeld_mut_hh",
    "heizkosten_m_hh",
    "grundr_bew_zeiten",
    "grundr_zeiten",
    "wohnfläche_hh",
    "bruttokaltmiete_m_hh",
    "hh_typ",
    "hh_id",
    "geburtstag",
    "mietstufe",
    "arbeitsl_monate_lfdj",
    "p_id",
    "schwerbeh_g",
    "arbeitsl_monate_vorj",
]


### Debug Mode

In addition to errors and warnings `compute_taxes_and_transfers` can also be used in debug mode by setting the argument `debug=True`. In this mode, the function returns all inputs and outputs that can be computed while issuing error messages for the parts where the code fails. It is thus a very useful tool to help you set up your code correctly and detect the sources of problems that might arise in the process. Check out the [troubleshooting tutorial](debugging.ipynb) for more information.

### Computing Child Benefits and Taxes

In this section we will compute the *Kindergeld* for our synthetic households. Since households can also claim a tax credit instead of the child benefit (the *Kinderfreibetrag*), we will also compute the income taxes for each household. By default, GETTSIM chooses the financially more favorable option for each case. The results will thus let us inspect how the policy affects different income levels in our data. 

#### Child Benefit

After ensuring all necessary inputs are defined using the features described above, we can compute the *Kindergeld* for our household `data` given the policy environment specified by `policy_params` and `policy_functions`. We compute the variables `kindergeld_m`, which indicates the *Kindergeld* on individual level, `kindergeld_m_tu`, which computes it on tax unit level, and `kindergeld_m_hh`, which computes it on household level.

In [37]:
kindergeld = compute_taxes_and_transfers(
    data=data,
    params=policy_params,
    targets=["kindergeld_m", "kindergeld_m_tu", "kindergeld_m_hh"],
    functions=policy_functions,
)


We can use the pandas function [pandas.Series.value_counts](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html) to assess the results. Computing the value frequencies in the data shows that the function assigns either no *Kindergeld* or €204 to individuals. In the data, the child benefit gets assigned to the child who it accounts for. As the `policy_params` shown above reveal, families receive €204 for the first and second child, €210 for the third, and €235 for every following child. Since the data includes only one- or two-child households, there are only two possible values for *Kindergeld* payments.

In [38]:
kindergeld.kindergeld_m.value_counts(normalize=True).round(2)

0.0      0.67
204.0    0.33
Name: kindergeld_m, dtype: float64

We can now look at the household and tax unit level to assess how much child benefits the families receive in total. In this example, the household id (`hh_id`) and tax unit id (`tu_id`) is identical for all households. However, even though the household types are distributed evenly in the data, the computed child benefit claims are not. A disproportionately large amount of tax units receives no child benefit even though there are children in the household. 

This difference stems from the variation in incomes. For higher income families, it is more advantageous to claim the *Kinderfreibetrag* which constitutes a tax credit. As GETTSIM by default always chooses the most beneficial taxes and transfers for households, it thus does not assign child benefits to the families for which the tax credit is more favorable.

In [39]:
kindergeld.kindergeld_m_tu.value_counts(normalize=True).round(2)

408.0    0.44
204.0    0.33
0.0      0.23
Name: kindergeld_m_tu, dtype: float64

In [40]:
kindergeld.kindergeld_m_hh.value_counts(normalize=True).round(2)

408.0    0.44
204.0    0.33
0.0      0.23
Name: kindergeld_m_hh, dtype: float64

#### Income Taxes

As the results above show, GETTSIM only computes the child benefit if it is more beneficial for the family than claiming the tax credit. To compare, we can additionally compute the income taxes for our data set `eink_st_tu`. We also compute the variable `bruttolohn_m_tu`, which gives the monthly gross income per tax unit (in our case, this is the combined income of the two adults in the household). 

In [41]:
income = compute_taxes_and_transfers(
    data=data,
    params=policy_params,
    targets=["eink_st_tu", "bruttolohn_m_tu"],
    functions=policy_functions,
)

For a better analysis of our data, we now create a column that indicates the number of children in a household. We can let GETTSIM do this for us by using ``anz_kinder_bis_17_hh`` as our target.

In [42]:
children = compute_taxes_and_transfers(
    data=data,
    params=policy_params,
    targets="anz_kinder_bis_17_hh",
    functions=policy_functions,
)

Since the gross income and child benefit per tax unit is computed on a monthly level while taxes are computed for the time unit of one year, we multiply the former by 12 and drop unused variables as well as duplicates from our DataFrame. The final DataFrame contains the yearly gross income, income tax, child benefit, and number of children in the household.

In [43]:
# Join results in one DataFrame.
df = kindergeld.join([income, children.anz_kinder_bis_17_hh]).set_index(data.tu_id)
# Multiply variables by 12 to generate yearly values.
df[["bruttolohn_tu", "kindergeld_tu"]] = df[["bruttolohn_m_tu", "kindergeld_m_tu"]] * 12
# Select variables of interest for further steps.
df = df[
    ["bruttolohn_tu", "eink_st_tu", "kindergeld_tu", "anz_kinder_bis_17_hh"]
].drop_duplicates()
df.head()

ValueError: Indexes have overlapping values: Index(['hh_id', 'tu_id', 'p_id'], dtype='object')

We can plot the results for our different household types. The plots below show how GETTSIM computes *Kindergeld* for our families. At a certain income level (around €80,000-€90,000) the tax credit becomes more favorable and GETTSIM assigns the tax break instead of the child benefit. 

In [ ]:
def plot_kindergeld(df):
    """Plot the child benefit and income taxes by household type. """
    plt.figure(figsize=[6, 10])
    for i in 0, 1, 2:
        # Create three subplots for each household type.
        plt.subplot(3, 1, i + 1)
        # Slice df according to household type.
        df_slice = df[df.anz_kinder_bis_17_hh == i].reset_index().drop_duplicates()
        # Plot income tax against gross income by tax unit.
        plt.plot(df_slice.bruttolohn_tu, df_slice["eink_st_tu"], label="Income tax")
        # Plot child benefit agains gross income by tax unit.
        plt.plot(
            df_slice.bruttolohn_tu, df_slice["kindergeld_tu"], label="Child benefit",
        )
        # Set title, axis labels, and legend.
        plt.title(f"{i} child(ren)")
        plt.ylabel("€ per year")
        plt.xlabel("Yearly gross income in € (per tax unit)")
        plt.legend()
        plt.tight_layout()

In [ ]:
plot_kindergeld(df)

### Columns Overriding Functions

Lastly, it is also possible to substitute internally computed variables using input columns in the data. To override an internal function, it is necessary to specify a column with the same name and pass it to `compute_taxes_and_transfers` using the argument `columns_overriding_functions`.

For instance, for this application we could override the internal function `_kindergeld_basis_m` and set the child benefit to 0. 

In [ ]:
new_data = data.copy()
new_data["kindergeld_basis_m"] = 0

Again, we compute the child benefit and income tax by tax unit. The argument `columns_overriding_functions` also accepts lists of columns to overwrite multiple functions.

In [ ]:
outputs = compute_taxes_and_transfers(
    data=new_data,
    params=policy_params,
    targets=["kindergeld_m_tu", "eink_st_tu", "bruttolohn_m_tu"],
    functions=policy_functions,
    columns_overriding_functions=["kindergeld_basis_m"],
)

In [ ]:
df_new = outputs.join(children["anz_kinder_bis_17_hh"]).set_index(new_data.tu_id)
df_new[["bruttolohn_tu", "kindergeld_tu"]] = (
    df_new[["bruttolohn_m_tu", "kindergeld_m_tu"]] * 12
)
df_new = df_new[
    ["bruttolohn_tu", "eink_st_tu", "kindergeld_tu", "anz_kinder_bis_17_hh"]
].drop_duplicates()

Since the child benefits are set to zero, GETTSIM computes the tax credit for all households instead. 

In [ ]:
plot_kindergeld(df_new)

Aside from overriding internal function outputs using data columns, it is also possible to substitute the functions entirely. Please refer to the [policy functions tutorial](policy_functions.ipynb) for more information.

#### Use Case for Columns Overriding Functions: Retirement Earnings

Retirement earnings (`ges_rente_m`) can be calculated by GETTSIM which requires several input variables including `entgeltp` or `grundr_zeiten`. 

However, in most data sets (e.g. the SOEP) retirement earnings are observed and those input variables are not. For some applications, it is, hence, more straight-forward to specify `columns_overriding_functions=["ges_rente_m"]` and use the measured retirement earnings directly. Then the pension-specific input variables like `entgeltp` or `grundr_zeiten` are not needed as input variables.

